# Modelo ETS (Error, Trend, Seasonality)

Este notebook implementa modelos ETS (Error, Trend, Seasonality) para predecir las ventas de dos productos, utilizando tres tipos de validación temporal:
- Walk-Forward Validation
- Rolling Window
- Expanding Window

Posteriormente, se utiliza Optimización Bayesiana para encontrar los mejores hiperparámetros y entrenar el modelo final para producción.


In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.exponential_smoothing.ets import ETSModel
import warnings
import optuna
warnings.filterwarnings('ignore')


In [2]:
# Cargar el dataset
df = pd.read_csv('data-set.csv', index_col=0)
print("Dataset cargado:")
print(f"Forma del dataset: {df.shape}")
print(f"\nPrimeras filas:")
print(df.head())
print(f"\nÚltimas filas:")
print(df.tail())
print(f"\nInformación del dataset:")
print(df.info())


Dataset cargado:
Forma del dataset: (127, 2)

Primeras filas:
    producto1   producto2
1  500.000000  200.000000
2  497.400893  210.686220
3  478.605317  222.018584
4  486.454125  233.920990
5  479.695678  238.402098

Últimas filas:
      producto1   producto2
123  164.610771  629.293034
124  150.881839  637.099467
125  151.788470  653.155282
126  137.047639  672.528345
127  141.990873  676.058092

Información del dataset:
<class 'pandas.core.frame.DataFrame'>
Index: 127 entries, 1 to 127
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   producto1  127 non-null    float64
 1   producto2  127 non-null    float64
dtypes: float64(2)
memory usage: 3.0 KB
None


## Función ETS (Error, Trend, Seasonality)

Implementación del modelo ETS que permite diferentes combinaciones de:
- **Error**: 'add' (aditivo) o 'mul' (multiplicativo)
- **Trend**: None, 'add' (aditivo) o 'mul' (multiplicativo)
- **Seasonality**: None, 'add' (aditivo) o 'mul' (multiplicativo)

El modelo ETS es más flexible que Holt-Winters ya que permite diferentes combinaciones de estos componentes.


In [3]:
def ets_forecast(data, error='add', trend=None, seasonal=None, seasonal_periods=12, forecast_horizon=1):
    """
    Calcula el modelo ETS y realiza predicciones.
    
    Parameters:
    -----------
    data : array-like
        Serie temporal de datos
    error : str
        Tipo de error: 'add' (aditivo) o 'mul' (multiplicativo)
    trend : str or None
        Tipo de tendencia: None, 'add' (aditivo) o 'mul' (multiplicativo)
    seasonal : str or None
        Tipo de estacionalidad: None, 'add' (aditivo) o 'mul' (multiplicativo)
    seasonal_periods : int
        Período estacional (por defecto 12 para datos mensuales con estacionalidad anual)
    forecast_horizon : int
        Número de períodos a predecir (por defecto 1)
    
    Returns:
    --------
    float or array
        Predicción(es) usando ETS
    """
    if len(data) < 2:
        return data[-1] if len(data) > 0 else 0
    
    # Si hay estacionalidad, necesitamos al menos 2 períodos estacionales
    if seasonal is not None and len(data) < 2 * seasonal_periods:
        # Si no hay suficientes datos para estacionalidad, usar modelo sin estacionalidad
        seasonal = None
    
    # Convertir a pandas Series
    data_series = pd.Series(data)
    
    try:
        # Crear y ajustar el modelo ETS
        model = ETSModel(
            data_series,
            error=error,
            trend=trend,
            seasonal=seasonal,
            seasonal_periods=seasonal_periods if seasonal is not None else None
        )
        
        # Ajustar el modelo
        fit = model.fit(disp=False)
        
        # Realizar predicción
        forecast = fit.forecast(steps=forecast_horizon)
        
        # Si solo se predice un paso, devolver un escalar
        if forecast_horizon == 1:
            return float(forecast.iloc[0]) if hasattr(forecast, 'iloc') else float(forecast[0])
        else:
            # Devolver array de predicciones
            if hasattr(forecast, 'values'):
                return forecast.values
            else:
                return np.array(forecast)
        
    except Exception as e:
        # En caso de error, usar el último valor como predicción
        print(f"Advertencia en ETS: {e}")
        return data[-1]


## Función 1: Walk-Forward Validation
    
En Walk-Forward Validation, se entrena el modelo con datos hasta un punto específico y se valida con el siguiente punto. Luego se avanza un paso y se repite el proceso.


In [4]:
def walk_forward_validation(dataset, error='add', trend=None, seasonal=None, seasonal_periods=12, train_size_min=24):
    """
    Realiza validación Walk-Forward para cada producto del dataset.
    
    Parameters:
    -----------
    dataset : pandas.DataFrame
        Dataset con las columnas de productos
    error : str
        Tipo de error para ETS: 'add' o 'mul'
    trend : str or None
        Tipo de tendencia para ETS: None, 'add' o 'mul'
    seasonal : str or None
        Tipo de estacionalidad para ETS: None, 'add' o 'mul'
    seasonal_periods : int
        Período estacional (por defecto 12)
    train_size_min : int
        Tamaño mínimo de entrenamiento antes de comenzar la validación (por defecto 24, mínimo 2*seasonal_periods)
    
    Returns:
    --------
    dict
        Diccionario con métricas para cada producto:
        - 'producto1': lista de diccionarios con métricas por iteración
        - 'producto2': lista de diccionarios con métricas por iteración
        Cada diccionario contiene: iteracion, ventana, rmse, mae
    """
    results = {}
    
    for producto in dataset.columns:
        print(f"\n=== Walk-Forward Validation para {producto} ===")
        data = dataset[producto].values
        n = len(data)
        
        metrics_list = []
        
        # Comenzar validación desde train_size_min hasta n-1
        for i in range(train_size_min, n):
            # Datos de entrenamiento: desde el inicio hasta i
            train_data = data[:i]
            # Dato de validación: i
            actual = data[i]
            
            # Realizar predicción
            try:
                prediction = ets_forecast(
                    train_data, 
                    error=error,
                    trend=trend,
                    seasonal=seasonal,
                    seasonal_periods=seasonal_periods,
                    forecast_horizon=1
                )
            except Exception as e:
                print(f"Error en iteración {i}: {e}")
                prediction = train_data[-1]  # Fallback al último valor
            
            # Calcular métricas
            rmse = np.sqrt(mean_squared_error([actual], [prediction]))
            mae = mean_absolute_error([actual], [prediction])
            
            metrics_list.append({
                'iteracion': i - train_size_min + 1,
                'ventana': i,
                'rmse': rmse,
                'mae': mae
            })
            
            if (i - train_size_min + 1) % 20 == 0:
                print(f"  Iteración {i - train_size_min + 1}/{n - train_size_min}: RMSE={rmse:.4f}, MAE={mae:.4f}")
        
        results[producto] = metrics_list
        print(f"  Total de iteraciones: {len(metrics_list)}")
    
    return results


## Función 2: Rolling Window Validation

En Rolling Window Validation, se utiliza una ventana fija de tamaño constante que se desliza a lo largo de la serie temporal.


In [5]:
def rolling_window_validation(dataset, error='add', trend=None, seasonal=None, seasonal_periods=12, train_window_size=24):
    """
    Realiza validación Rolling Window para cada producto del dataset.
    
    Parameters:
    -----------
    dataset : pandas.DataFrame
        Dataset con las columnas de productos
    error : str
        Tipo de error para ETS: 'add' o 'mul'
    trend : str or None
        Tipo de tendencia para ETS: None, 'add' o 'mul'
    seasonal : str or None
        Tipo de estacionalidad para ETS: None, 'add' o 'mul'
    seasonal_periods : int
        Período estacional (por defecto 12)
    train_window_size : int
        Tamaño fijo de la ventana de entrenamiento (por defecto 24, mínimo 2*seasonal_periods)
    
    Returns:
    --------
    dict
        Diccionario con métricas para cada producto:
        - 'producto1': lista de diccionarios con métricas por iteración
        - 'producto2': lista de diccionarios con métricas por iteración
        Cada diccionario contiene: iteracion, ventana, rmse, mae
    """
    results = {}
    
    for producto in dataset.columns:
        print(f"\n=== Rolling Window Validation para {producto} ===")
        data = dataset[producto].values
        n = len(data)
        
        metrics_list = []
        
        # Comenzar desde train_window_size hasta n-1
        for i in range(train_window_size, n):
            # Datos de entrenamiento: ventana fija de tamaño train_window_size
            train_data = data[i - train_window_size:i]
            # Dato de validación: i
            actual = data[i]
            
            # Realizar predicción
            try:
                prediction = ets_forecast(
                    train_data, 
                    error=error,
                    trend=trend,
                    seasonal=seasonal,
                    seasonal_periods=seasonal_periods,
                    forecast_horizon=1
                )
            except Exception as e:
                print(f"Error en iteración {i}: {e}")
                prediction = train_data[-1]  # Fallback al último valor
            
            # Calcular métricas
            rmse = np.sqrt(mean_squared_error([actual], [prediction]))
            mae = mean_absolute_error([actual], [prediction])
            
            metrics_list.append({
                'iteracion': i - train_window_size + 1,
                'ventana': f"{i - train_window_size}-{i}",
                'rmse': rmse,
                'mae': mae
            })
            
            if (i - train_window_size + 1) % 20 == 0:
                print(f"  Iteración {i - train_window_size + 1}/{n - train_window_size}: RMSE={rmse:.4f}, MAE={mae:.4f}")
        
        results[producto] = metrics_list
        print(f"  Total de iteraciones: {len(metrics_list)}")
    
    return results


## Función 3: Expanding Window Validation

En Expanding Window Validation, la ventana de entrenamiento crece con cada iteración, comenzando desde un tamaño mínimo y expandiéndose hasta incluir todos los datos disponibles hasta ese punto.


In [6]:
def expanding_window_validation(dataset, error='add', trend=None, seasonal=None, seasonal_periods=12, train_size_min=24):
    """
    Realiza validación Expanding Window para cada producto del dataset.
    
    Parameters:
    -----------
    dataset : pandas.DataFrame
        Dataset con las columnas de productos
    error : str
        Tipo de error para ETS: 'add' o 'mul'
    trend : str or None
        Tipo de tendencia para ETS: None, 'add' o 'mul'
    seasonal : str or None
        Tipo de estacionalidad para ETS: None, 'add' o 'mul'
    seasonal_periods : int
        Período estacional (por defecto 12)
    train_size_min : int
        Tamaño mínimo inicial de la ventana de entrenamiento (por defecto 24, mínimo 2*seasonal_periods)
    
    Returns:
    --------
    dict
        Diccionario con métricas para cada producto:
        - 'producto1': lista de diccionarios con métricas por iteración
        - 'producto2': lista de diccionarios con métricas por iteración
        Cada diccionario contiene: iteracion, ventana, rmse, mae
    """
    results = {}
    
    for producto in dataset.columns:
        print(f"\n=== Expanding Window Validation para {producto} ===")
        data = dataset[producto].values
        n = len(data)
        
        metrics_list = []
        
        # Comenzar validación desde train_size_min hasta n-1
        for i in range(train_size_min, n):
            # Datos de entrenamiento: desde el inicio hasta i (ventana que se expande)
            train_data = data[:i]
            # Dato de validación: i
            actual = data[i]
            
            # Realizar predicción
            try:
                prediction = ets_forecast(
                    train_data, 
                    error=error,
                    trend=trend,
                    seasonal=seasonal,
                    seasonal_periods=seasonal_periods,
                    forecast_horizon=1
                )
            except Exception as e:
                print(f"Error en iteración {i}: {e}")
                prediction = train_data[-1]  # Fallback al último valor
            
            # Calcular métricas
            rmse = np.sqrt(mean_squared_error([actual], [prediction]))
            mae = mean_absolute_error([actual], [prediction])
            
            metrics_list.append({
                'iteracion': i - train_size_min + 1,
                'ventana': i,  # Tamaño de la ventana que se expande
                'rmse': rmse,
                'mae': mae
            })
            
            if (i - train_size_min + 1) % 20 == 0:
                print(f"  Iteración {i - train_size_min + 1}/{n - train_size_min}: RMSE={rmse:.4f}, MAE={mae:.4f}")
        
        results[producto] = metrics_list
        print(f"  Total de iteraciones: {len(metrics_list)}")
    
    return results


## Ejecución de las Validaciones

Después de definir las funciones que permitirán ejecutar el modelo teniendo en cuenta los tres tipos de validación, ahora ejecutamos las tres validaciones para cada producto y calculamos las métricas promedio.

Para ETS, inicialmente usaremos un modelo automático que permite que statsmodels seleccione el mejor tipo de error, tendencia y estacionalidad.


In [7]:
# Parámetros de validación
SEASONAL_PERIODS = 12  # Período estacional (asumiendo datos mensuales con estacionalidad anual)
TRAIN_SIZE_MIN = 24  # Tamaño mínimo de entrenamiento (2 * seasonal_periods)
TRAIN_WINDOW_SIZE = 24  # Tamaño fijo para Rolling Window

# Para la validación inicial, usaremos ETS automático (dejando que statsmodels seleccione)
# Usaremos error='add', trend='add', seasonal='add' como configuración inicial
# En la optimización bayesiana exploraremos todas las combinaciones

# Ejecutar Walk-Forward Validation
print("=" * 60)
print("WALK-FORWARD VALIDATION")
print("=" * 60)
results_wf = walk_forward_validation(
    df, 
    error='add',
    trend='add',
    seasonal='add',
    seasonal_periods=SEASONAL_PERIODS,
    train_size_min=TRAIN_SIZE_MIN
)


WALK-FORWARD VALIDATION

=== Walk-Forward Validation para producto1 ===
  Iteración 20/103: RMSE=10.5308, MAE=10.5308
  Iteración 40/103: RMSE=10.5419, MAE=10.5419
  Iteración 60/103: RMSE=10.9976, MAE=10.9976
  Iteración 80/103: RMSE=3.4179, MAE=3.4179
  Iteración 100/103: RMSE=9.5698, MAE=9.5698
  Total de iteraciones: 103

=== Walk-Forward Validation para producto2 ===
  Iteración 20/103: RMSE=25.0076, MAE=25.0076
  Iteración 40/103: RMSE=10.4811, MAE=10.4811
  Iteración 60/103: RMSE=14.6686, MAE=14.6686
  Iteración 80/103: RMSE=11.9310, MAE=11.9310
  Iteración 100/103: RMSE=11.4784, MAE=11.4784
  Total de iteraciones: 103


In [8]:
# Ejecutar Rolling Window Validation
print("\n" + "=" * 60)
print("ROLLING WINDOW VALIDATION")
print("=" * 60)
results_rw = rolling_window_validation(
    df, 
    error='add',
    trend='add',
    seasonal='add',
    seasonal_periods=SEASONAL_PERIODS,
    train_window_size=TRAIN_WINDOW_SIZE
)



ROLLING WINDOW VALIDATION

=== Rolling Window Validation para producto1 ===
  Iteración 20/103: RMSE=8.6837, MAE=8.6837
  Iteración 40/103: RMSE=4.0652, MAE=4.0652
  Iteración 60/103: RMSE=12.7746, MAE=12.7746
  Iteración 80/103: RMSE=0.6420, MAE=0.6420
  Iteración 100/103: RMSE=0.9298, MAE=0.9298
  Total de iteraciones: 103

=== Rolling Window Validation para producto2 ===
  Iteración 20/103: RMSE=7.4544, MAE=7.4544
  Iteración 40/103: RMSE=7.7448, MAE=7.7448
  Iteración 60/103: RMSE=13.2633, MAE=13.2633
  Iteración 80/103: RMSE=21.4691, MAE=21.4691
  Iteración 100/103: RMSE=9.1008, MAE=9.1008
  Total de iteraciones: 103


In [9]:
# Ejecutar Expanding Window Validation
print("\n" + "=" * 60)
print("EXPANDING WINDOW VALIDATION")
print("=" * 60)
results_ew = expanding_window_validation(
    df, 
    error='add',
    trend='add',
    seasonal='add',
    seasonal_periods=SEASONAL_PERIODS,
    train_size_min=TRAIN_SIZE_MIN
)



EXPANDING WINDOW VALIDATION

=== Expanding Window Validation para producto1 ===
  Iteración 20/103: RMSE=10.5308, MAE=10.5308
  Iteración 40/103: RMSE=10.5419, MAE=10.5419
  Iteración 60/103: RMSE=10.9976, MAE=10.9976
  Iteración 80/103: RMSE=3.4179, MAE=3.4179
  Iteración 100/103: RMSE=9.5698, MAE=9.5698
  Total de iteraciones: 103

=== Expanding Window Validation para producto2 ===
  Iteración 20/103: RMSE=25.0076, MAE=25.0076
  Iteración 40/103: RMSE=10.4811, MAE=10.4811
  Iteración 60/103: RMSE=14.6686, MAE=14.6686
  Iteración 80/103: RMSE=11.9310, MAE=11.9310
  Iteración 100/103: RMSE=11.4784, MAE=11.4784
  Total de iteraciones: 103


In [10]:
def calcular_metricas_promedio(results, nombre_validacion):
    """
    Calcula las métricas promedio para cada producto.
    
    Parameters:
    -----------
    results : dict
        Resultados de la validación
    nombre_validacion : str
        Nombre del tipo de validación
    
    Returns:
    --------
    dict
        Diccionario con métricas promedio por producto
    """
    metricas_promedio = {}
    
    print(f"\n{'='*60}")
    print(f"MÉTRICAS PROMEDIO - {nombre_validacion.upper()}")
    print(f"{'='*60}")
    
    for producto in results.keys():
        metrics_list = results[producto]
        
        # Calcular promedios
        rmse_promedio = np.mean([m['rmse'] for m in metrics_list])
        mae_promedio = np.mean([m['mae'] for m in metrics_list])
        
        metricas_promedio[producto] = {
            'rmse_promedio': rmse_promedio,
            'mae_promedio': mae_promedio,
            'num_iteraciones': len(metrics_list)
        }
        
        print(f"\n{producto}:")
        print(f"  RMSE Promedio: {rmse_promedio:.4f}")
        print(f"  MAE Promedio: {mae_promedio:.4f}")
        print(f"  Número de iteraciones: {len(metrics_list)}")
    
    return metricas_promedio

# Calcular métricas promedio para cada tipo de validación
metricas_wf = calcular_metricas_promedio(results_wf, "Walk-Forward")
metricas_rw = calcular_metricas_promedio(results_rw, "Rolling Window")
metricas_ew = calcular_metricas_promedio(results_ew, "Expanding Window")



MÉTRICAS PROMEDIO - WALK-FORWARD

producto1:
  RMSE Promedio: 7.7732
  MAE Promedio: 7.7732
  Número de iteraciones: 103

producto2:
  RMSE Promedio: 13.1844
  MAE Promedio: 13.1844
  Número de iteraciones: 103

MÉTRICAS PROMEDIO - ROLLING WINDOW

producto1:
  RMSE Promedio: 8.7473
  MAE Promedio: 8.7473
  Número de iteraciones: 103

producto2:
  RMSE Promedio: 16.4938
  MAE Promedio: 16.4938
  Número de iteraciones: 103

MÉTRICAS PROMEDIO - EXPANDING WINDOW

producto1:
  RMSE Promedio: 7.7732
  MAE Promedio: 7.7732
  Número de iteraciones: 103

producto2:
  RMSE Promedio: 13.1844
  MAE Promedio: 13.1844
  Número de iteraciones: 103


## Comparación de Resultados y Selección del Mejor Método de Validación

Comparamos los RMSE promedio de cada tipo de validación para seleccionar el mejor método.


In [11]:
# Crear DataFrame comparativo
comparacion = []

for producto in df.columns:
    comparacion.append({
        'Producto': producto,
        'Validacion': 'Walk-Forward',
        'RMSE_Promedio': metricas_wf[producto]['rmse_promedio'],
        'MAE_Promedio': metricas_wf[producto]['mae_promedio']
    })
    comparacion.append({
        'Producto': producto,
        'Validacion': 'Rolling Window',
        'RMSE_Promedio': metricas_rw[producto]['rmse_promedio'],
        'MAE_Promedio': metricas_rw[producto]['mae_promedio']
    })
    comparacion.append({
        'Producto': producto,
        'Validacion': 'Expanding Window',
        'RMSE_Promedio': metricas_ew[producto]['rmse_promedio'],
        'MAE_Promedio': metricas_ew[producto]['mae_promedio']
    })

df_comparacion = pd.DataFrame(comparacion)
print("\n" + "=" * 80)
print("COMPARACIÓN DE MÉTODOS DE VALIDACIÓN")
print("=" * 80)
print(df_comparacion.to_string(index=False))

# Identificar el mejor método para cada producto
print("\n" + "=" * 80)
print("MEJOR MÉTODO DE VALIDACIÓN POR PRODUCTO (basado en RMSE Promedio)")
print("=" * 80)
mejor_validacion = {}
for producto in df.columns:
    producto_df = df_comparacion[df_comparacion['Producto'] == producto]
    mejor = producto_df.loc[producto_df['RMSE_Promedio'].idxmin()]
    mejor_validacion[producto] = mejor['Validacion']
    print(f"\n{producto}:")
    print(f"  Mejor método: {mejor['Validacion']}")
    print(f"  RMSE Promedio: {mejor['RMSE_Promedio']:.4f}")



COMPARACIÓN DE MÉTODOS DE VALIDACIÓN
 Producto       Validacion  RMSE_Promedio  MAE_Promedio
producto1     Walk-Forward       7.773178      7.773178
producto1   Rolling Window       8.747306      8.747306
producto1 Expanding Window       7.773178      7.773178
producto2     Walk-Forward      13.184430     13.184430
producto2   Rolling Window      16.493841     16.493841
producto2 Expanding Window      13.184430     13.184430

MEJOR MÉTODO DE VALIDACIÓN POR PRODUCTO (basado en RMSE Promedio)

producto1:
  Mejor método: Walk-Forward
  RMSE Promedio: 7.7732

producto2:
  Mejor método: Walk-Forward
  RMSE Promedio: 13.1844


## Optimización Bayesiana para Encontrar los Mejores Parámetros ETS

Utilizamos Optimización Bayesiana (Optuna) para encontrar los mejores parámetros de ETS (error, trend, seasonal) para cada producto, usando el mejor método de validación identificado.


In [12]:
def objective_walk_forward(trial, data, seasonal_periods=12, train_size_min=24):
    """
    Función objetivo para optimización bayesiana usando Walk-Forward Validation.
    """
    error = trial.suggest_categorical('error', ['add', 'mul'])
    trend = trial.suggest_categorical('trend', [None, 'add', 'mul'])
    seasonal = trial.suggest_categorical('seasonal', [None, 'add', 'mul'])
    
    n = len(data)
    errors = []
    
    for i in range(train_size_min, n):
        train_data = data[:i]
        actual = data[i]
        
        try:
            prediction = ets_forecast(
                train_data, 
                error=error,
                trend=trend,
                seasonal=seasonal,
                seasonal_periods=seasonal_periods,
                forecast_horizon=1
            )
            error_val = np.sqrt(mean_squared_error([actual], [prediction]))
            if np.isfinite(error_val):
                errors.append(error_val)
        except:
            continue
    
    if len(errors) == 0:
        return float('inf')
    
    return np.mean(errors)

def objective_rolling_window(trial, data, seasonal_periods=12, train_window_size=24):
    """
    Función objetivo para optimización bayesiana usando Rolling Window Validation.
    """
    error = trial.suggest_categorical('error', ['add', 'mul'])
    trend = trial.suggest_categorical('trend', [None, 'add', 'mul'])
    seasonal = trial.suggest_categorical('seasonal', [None, 'add', 'mul'])
    
    n = len(data)
    errors = []
    
    for i in range(train_window_size, n):
        train_data = data[i - train_window_size:i]
        actual = data[i]
        
        try:
            prediction = ets_forecast(
                train_data, 
                error=error,
                trend=trend,
                seasonal=seasonal,
                seasonal_periods=seasonal_periods,
                forecast_horizon=1
            )
            error_val = np.sqrt(mean_squared_error([actual], [prediction]))
            if np.isfinite(error_val):
                errors.append(error_val)
        except:
            continue
    
    if len(errors) == 0:
        return float('inf')
    
    return np.mean(errors)

def objective_expanding_window(trial, data, seasonal_periods=12, train_size_min=24):
    """
    Función objetivo para optimización bayesiana usando Expanding Window Validation.
    """
    error = trial.suggest_categorical('error', ['add', 'mul'])
    trend = trial.suggest_categorical('trend', [None, 'add', 'mul'])
    seasonal = trial.suggest_categorical('seasonal', [None, 'add', 'mul'])
    
    n = len(data)
    errors = []
    
    for i in range(train_size_min, n):
        train_data = data[:i]
        actual = data[i]
        
        try:
            prediction = ets_forecast(
                train_data, 
                error=error,
                trend=trend,
                seasonal=seasonal,
                seasonal_periods=seasonal_periods,
                forecast_horizon=1
            )
            error_val = np.sqrt(mean_squared_error([actual], [prediction]))
            if np.isfinite(error_val):
                errors.append(error_val)
        except:
            continue
    
    if len(errors) == 0:
        return float('inf')
    
    return np.mean(errors)


In [13]:
# Realizar optimización bayesiana para cada producto usando su mejor método de validación
print("\n" + "=" * 80)
print("OPTIMIZACIÓN BAYESIANA")
print("=" * 80)

mejores_parametros = {}

for producto in df.columns:
    print(f"\n--- Optimizando {producto} ---")
    data = df[producto].values
    mejor_metodo = mejor_validacion[producto]
    
    print(f"{producto}: Mejor método = {mejor_metodo}")
    
    # Seleccionar función objetivo según el mejor método
    if mejor_metodo == 'Walk-Forward':
        objective_func = lambda trial: objective_walk_forward(trial, data, SEASONAL_PERIODS, TRAIN_SIZE_MIN)
    elif mejor_metodo == 'Rolling Window':
        objective_func = lambda trial: objective_rolling_window(trial, data, SEASONAL_PERIODS, TRAIN_WINDOW_SIZE)
    else:  # Expanding Window
        objective_func = lambda trial: objective_expanding_window(trial, data, SEASONAL_PERIODS, TRAIN_SIZE_MIN)
    
    # Crear estudio de Optuna
    study = optuna.create_study(direction='minimize')
    study.optimize(objective_func, n_trials=50, show_progress_bar=True)
    
    # Obtener mejores parámetros
    best_params = study.best_params
    best_rmse = study.best_value
    
    mejores_parametros[producto] = {
        'error': best_params['error'],
        'trend': best_params['trend'],
        'seasonal': best_params['seasonal'],
        'rmse': best_rmse,
        'metodo': mejor_metodo
    }
    
    print(f"  Mejor error: {best_params['error']}")
    print(f"  Mejor trend: {best_params['trend']}")
    print(f"  Mejor seasonal: {best_params['seasonal']}")
    print(f"  Mejor RMSE: {best_rmse:.4f}")
    print(f"  Método usado: {mejor_metodo}")


[I 2025-11-30 13:41:02,331] A new study created in memory with name: no-name-733ba57a-c023-4243-97ad-35f7344ea06b



OPTIMIZACIÓN BAYESIANA

--- Optimizando producto1 ---
producto1: Mejor método = Walk-Forward


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-11-30 13:41:09,585] Trial 0 finished with value: 8.8311470777312 and parameters: {'error': 'add', 'trend': None, 'seasonal': 'mul'}. Best is trial 0 with value: 8.8311470777312.
[I 2025-11-30 13:41:47,760] Trial 1 finished with value: 8.039064086808722 and parameters: {'error': 'add', 'trend': 'mul', 'seasonal': 'add'}. Best is trial 1 with value: 8.039064086808722.
[I 2025-11-30 13:41:55,814] Trial 2 finished with value: 8.83690640333698 and parameters: {'error': 'mul', 'trend': None, 'seasonal': 'mul'}. Best is trial 1 with value: 8.039064086808722.
[I 2025-11-30 13:41:56,457] Trial 3 finished with value: 8.197348533726196 and parameters: {'error': 'mul', 'trend': None, 'seasonal': None}. Best is trial 1 with value: 8.039064086808722.
[I 2025-11-30 13:41:57,101] Trial 4 finished with value: 8.197348533726196 and parameters: {'error': 'mul', 'trend': None, 'seasonal': None}. Best is trial 1 with value: 8.039064086808722.
[I 2025-11-30 13:42:41,531] Trial 5 finished with value:

[I 2025-11-30 13:47:43,230] A new study created in memory with name: no-name-0fd31ce5-9c61-465a-8f0a-328cd7406b7d


[I 2025-11-30 13:47:43,228] Trial 49 finished with value: 6.81782816148952 and parameters: {'error': 'add', 'trend': 'mul', 'seasonal': None}. Best is trial 8 with value: 6.69881404918342.
  Mejor error: mul
  Mejor trend: mul
  Mejor seasonal: None
  Mejor RMSE: 6.6988
  Método usado: Walk-Forward

--- Optimizando producto2 ---
producto2: Mejor método = Walk-Forward


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-11-30 13:47:51,926] Trial 0 finished with value: 15.80706384729923 and parameters: {'error': 'mul', 'trend': None, 'seasonal': 'mul'}. Best is trial 0 with value: 15.80706384729923.
[I 2025-11-30 13:47:53,552] Trial 1 finished with value: 12.731160061649007 and parameters: {'error': 'mul', 'trend': 'add', 'seasonal': None}. Best is trial 1 with value: 12.731160061649007.
[I 2025-11-30 13:48:05,180] Trial 2 finished with value: 15.088354950986533 and parameters: {'error': 'add', 'trend': None, 'seasonal': 'add'}. Best is trial 1 with value: 12.731160061649007.
[I 2025-11-30 13:48:32,519] Trial 3 finished with value: 12.965108991366622 and parameters: {'error': 'mul', 'trend': 'add', 'seasonal': 'add'}. Best is trial 1 with value: 12.731160061649007.
[I 2025-11-30 13:48:33,228] Trial 4 finished with value: 14.322651888189824 and parameters: {'error': 'add', 'trend': None, 'seasonal': None}. Best is trial 1 with value: 12.731160061649007.
[I 2025-11-30 13:48:35,476] Trial 5 finish

## Modelo Final para Producción

Entrenamos el modelo final con todos los datos disponibles usando los mejores parámetros encontrados y realizamos la predicción para el siguiente mes (1 pronóstico).


In [14]:
# Entrenar modelo final y realizar predicción para producción
print("\n" + "=" * 80)
print("PREDICCIÓN FINAL PARA PRODUCCIÓN")
print("=" * 80)

predicciones_produccion = {}

for producto in df.columns:
    print(f"\n--- {producto} ---")
    data = df[producto].values
    
    # Obtener mejores parámetros
    params = mejores_parametros[producto]
    
    print(f"Parámetros del modelo:")
    print(f"  Error: {params['error']}")
    print(f"  Trend: {params['trend']}")
    print(f"  Seasonal: {params['seasonal']}")
    print(f"  RMSE en validación: {params['rmse']:.4f}")
    
    # Entrenar modelo final con todos los datos
    try:
        prediccion = ets_forecast(
            data,
            error=params['error'],
            trend=params['trend'],
            seasonal=params['seasonal'],
            seasonal_periods=SEASONAL_PERIODS,
            forecast_horizon=1
        )
        
        predicciones_produccion[producto] = prediccion
        
        print(f"  Predicción para el siguiente mes: {prediccion:.4f}")
        print(f"  Último valor observado: {data[-1]:.4f}")
        
    except Exception as e:
        print(f"  Error al generar predicción: {e}")
        # Fallback: usar último valor
        predicciones_produccion[producto] = data[-1]
        print(f"  Predicción (fallback): {data[-1]:.4f}")

# Mostrar resumen de predicciones
print("\n" + "=" * 80)
print("RESUMEN DE PREDICCIONES PARA PRODUCCIÓN")
print("=" * 80)
for producto, prediccion in predicciones_produccion.items():
    print(f"{producto}: {prediccion:.4f}")



PREDICCIÓN FINAL PARA PRODUCCIÓN

--- producto1 ---
Parámetros del modelo:
  Error: mul
  Trend: mul
  Seasonal: None
  RMSE en validación: 6.6988
  Predicción para el siguiente mes: 133.2129
  Último valor observado: 141.9909

--- producto2 ---
Parámetros del modelo:
  Error: add
  Trend: add
  Seasonal: None
  RMSE en validación: 12.0211
  Predicción para el siguiente mes: 692.1065
  Último valor observado: 676.0581

RESUMEN DE PREDICCIONES PARA PRODUCCIÓN
producto1: 133.2129
producto2: 692.1065
